In [ ]:
# Utils
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchvision import transforms
import torchvision
! pip install tensorboardX
from tensorboardX import SummaryWriter

# sklearn
from sklearn.metrics import confusion_matrix, classification_report

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 731.9 kB/s eta 0:00:00


In [ ]:
batch_size=128
epochs=50
transform=transforms.Compose([transforms.Resize((256,256)),
                              transforms.CenterCrop(224),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
#transforms.Compose()는 여러 변환을 하나로 묶어 한번에 변환을 시켜줌.
#transforms.ToTensor() PIL이미지나 Numpy배열을 Pytorch 텐서로 변환하여 이미지를 네트워크 입력으로 사용할 수 있게함
#transforms.Normalize()각 픽셀 값을 정규화하여, 각 채널의 평균과 표준 편차를 0.5,0.5,0.5로 설정하여
#평균이 0이되고 표준편차가 1이되도록 이미지를 정규화함.
#이를 통해 픽셀 값 범위를 [-0.5,0.5]에서 [-1,1]매핑함.
train_set=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
train_loader=torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=2)
test_set=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
test_loader=torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=False,num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print(len(train_set))
print(len(test_set))

50000
10000


In [ ]:
class VGGNet11(nn.Module):
  def __init__(self):
    super().__init__()

    self.Conv_layer=nn.Sequential(
        nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),


        nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)

        )
    self.fc=nn.Sequential(
        nn.Linear(in_features=512*7*7,out_features=512),
        nn.ReLU(),
        nn.Dropout(p=0.5),

        nn.Linear(in_features=512,out_features=256),
        nn.ReLU(),
        nn.Dropout(p=0.5),

        nn.Linear(in_features=256,out_features=10),
        nn.Softmax(dim=1)

    )
    self.apply(self._init_weight_bias)

  def _init_weight_bias(self,module):
    classname=module.__class__.__name__
    if classname=='Conv2d':
      nn.init.normal_(module.weight,mean=0,std=0.01)
      nn.init.constant_(module.bias,val=0)
    elif classname=='Linear':
        nn.init.normal_(module.weight,mean=0,std=0.01)
        nn.init.constant_(module.bias,val=0)

  def forward(self,x):
    x=self.Conv_layer(x)
    x=x.view(x.size(0),-1)
    x=self.fc(x)

    return x

Set Device and Check Summary

In [ ]:
use_cuda=torch.cuda.is_available()
print("use_cuda:",use_cuda)

FloatTensor= torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
device=torch.device("cuda:0" if use_cuda else "Cpu")
VGGNet=VGGNet11().to(device)

X=torch.randn(size=(3,224,224)).type(FloatTensor)
print(summary(VGGNet,(3,224,224)))

use_cuda: True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14    

#Loss and optimizer

In [ ]:
batch_size=256
learning_rate=0.0001
epochs=74
model=VGGNet11().to(device)
criterion=F.cross_entropy
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

Training and Testing

In [ ]:
writer=SummaryWriter("./vggnet/tensorboard")


In [ ]:
def train(model,device,train_loader,optimizer,epoch):
  model.train()
  for batch_size, (data,target)in enumerate(train_loader):
    target=target.type(torch.LongTensor)
    data,target=data.to(device),target.to(device)
    optimizer.zero_grad()
    output=model(data)
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()
    if batch_size % 30 == 0:
            print(f"{batch_size*len(data)}/{len(train_loader.dataset)}")

In [ ]:
def test(model,device,test_loader):
  model.eval()
  test_loss=0
  correct=0
  with torch.no_grad():
    for data, target in test_loader:
      data,target=data.to(device),target.to(device)
      output=model(data)
      test_loss += criterion(output,target,reduction='mean').item()
      writer.add_scalar("Test Loss",test_loss,epochs)
      pred=output.argmax(1)
      correct+=float((pred==target).sum())
      writer.add_scalar("Test Accuracy",correct,epochs)

    test_loss/=len(test_loader.dataset)
    correct/=len(test_loader.dataset)
    return test_loss,correct
    writer.close()

Per-epoch Activity

In [ ]:
for epoch in tqdm(range(1,epochs+1)):
  train(model,device,train_loader,optimizer,epoch)
  test_loss,test_accuracy=test(model,device,test_loader)
  writer.add_scalar("Test Loss",test_loss,epoch)
  writer.add_scalar("Test Accuracy",test_accuracy,epoch)
  print(f"Processing Result = Epoch: {epoch} Loss: {test_loss} Accuracy: {test_accuracy}")
  writer.close

  0%|          | 0/50 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0/50000
3840/50000
7680/50000
11520/50000
15360/50000
19200/50000
23040/50000
26880/50000
30720/50000
34560/50000
38400/50000
42240/50000
46080/50000


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


31200/50000


  2%|▏         | 1/50 [05:45<4:42:23, 345.79s/it]

...

Processing Result = Epoch: 6 Loss: 0.0155676677942276 Accuracy: 0.486
0/50000
3840/50000
7680/50000
11520/50000
15360/50000
19200/50000
23040/50000
26880/50000
30720/50000
34560/50000
38400/50000
42240/50000
46080/50000
31200/50000


 14%|█▍        | 7/50 [41:01<4:12:27, 352.28s/it]

Processing Result = Epoch: 7 Loss: 0.015406526792049407 Accuracy: 0.5092
0/50000
3840/50000
7680/50000
11520/50000
15360/50000
19200/50000
23040/50000
26880/50000
30720/50000
34560/50000
38400/50000
42240/50000
46080/50000
31200/50000


 16%|█▌        | 8/50 [46:53<4:06:32, 352.19s/it]

Processing Result = Epoch: 8 Loss: 0.015396364116668702 Accuracy: 0.5088
0/50000
3840/50000
7680/50000


Visualization of VGG Net 11


*   Accuracy and Loss




In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./vggnet/tensorboard --port=8008